In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
     

Data Gathering

Download data from a given URL.

In [1]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
!unzip -q traffic-signs-data.zip
 

--2022-12-13 10:56:06--  https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.249.90.229, 13.249.90.192, 13.249.90.90, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.249.90.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  60.3MB/s    in 2.0s    

2022-12-13 10:56:08 (60.3 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]



Read data from pickle files.

In [2]:
import pickle

train_data = pickle.load(open("train.p", "rb"))
val_data = pickle.load(open("valid.p", "rb"))
test_data = pickle.load(open("test.p", "rb"))
     

get_features_labels = lambda x : (x["features"], x["labels"])
X_train, y_train = get_features_labels(train_data)
X_val, y_val = get_features_labels(val_data)
X_test, y_test = get_features_labels(test_data)

Define the input shape

In [3]:
num_train_samples, *input_shape = X_train.shape
num_classes = 43
print("Input shape:", input_shape)

Input shape: [32, 32, 3]


Since the pixel lies between 0-255, need scaling -> would add a rescaling layer later in the model.



In [6]:
print("Pixel range:", np.min(X_train), np.max(X_train))


Pixel range: 0 255


Data Shuffling


In [7]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)
X_test, y_test = shuffle(X_test, y_test)

Data Augmentation


In [8]:
BATCH_SIZE = 128
     

from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmentator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2, 
    horizontal_flip=True
)

ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
ds_train_aug = augmentator.flow(X_train, y_train, batch_size=BATCH_SIZE)

Model Architecture


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [10]:
model = Sequential()

model.add(Rescaling(scale=1./255, input_shape=input_shape))

model.add(Conv2D(filters=32, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (16 x 16 x 32)

model.add(Conv2D(filters=64, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (8 x 8 x 64)

model.add(Flatten())                                                                 # Input shape: (4096)
model.add(Dense(512, activation="relu"))                                             # Input shape: (512)
model.add(Dense(num_classes, activation="softmax"))                                  # Input shape: (num_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        416       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4128      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0

In [11]:
rmsprop_opt = tf.optimizers.RMSprop()
model.compile(loss="sparse_categorical_crossentropy", optimizer=rmsprop_opt, metrics=["acc"])


Training


In [13]:
history = model.fit(ds_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
272/272 [==============================] - 154s 567ms/step - loss: 0.1383 - acc: 0.9596 - val_loss: 1.6227 - val_acc: 0.7120
Epoch 2/10
272/272 [==============================] - 155s 571ms/step - loss: 0.0533 - acc: 0.9869 - val_loss: 0.6176 - val_acc: 0.9023
Epoch 3/10
272/272 [==============================] - 153s 564ms/step - loss: 0.0309 - acc: 0.9922 - val_loss: 0.6322 - val_acc: 0.9249
Epoch 4/10
272/272 [==============================] - 153s 564ms/step - loss: 0.0252 - acc: 0.9941 - val_loss: 0.6583 - val_acc: 0.9091
Epoch 5/10
272/272 [==============================] - 153s 563ms/step - loss: 0.0185 - acc: 0.9958 - val_loss: 0.7733 - val_acc: 0.9102
Epoch 6/10
272/272 [==============================] - 154s 565ms/step - loss: 0.0155 - acc: 0.9966 - val_loss: 0.7824 - val_acc: 0.9252
Epoch 7/10
272/272 [==============================] - 154s 565ms/step - loss: 0.0136 - acc: 0.9970 - val_loss: 0.4636 - val_acc: 0.9354
Epoch 8/10
272/272 [============================

In [14]:
model.save("traffic_sign.h5")


Evaluation


In [15]:
saved_model = tf.keras.models.load_model("traffic_sign.h5")
     

saved_model.evaluate(X_test, y_test)
     

395/395 [==============================] - 15s 37ms/step - loss: 0.6376 - acc: 0.9380


[0.6375763416290283, 0.938004732131958]